# Metadata

```
Course:   DS5001
Module:   098 Lab
Topic:    Gibbs Sampler
Author:   R.C. Alvarado

Purpose:  We develop an LDA topic modeler using collapsed Gibbs sample as described by [Griffiths and Steyvers (2004)].
```

## Setup

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from nltk.corpus import stopwords 

## Functions

### Convert Corpus 

We convert the list of token lists (DOC) into TOKEN and VOCAB tables.

In [2]:
class Corpus(): 
    
    def __init__(self, doclist):

        self.docs = doclist
        
        # Create DOC table from F1 doclist
        self.DOC = pd.DataFrame(doclist, columns=['doc_str'])
        self.DOC.index.name = 'doc_id'
        self.DOC

        # Convert docs into tokens
        stop_words = set(stopwords.words('english')) 
        tokens = []
        for i, doc in enumerate(doclist):
            for j, token in enumerate(doc.split()):
                term_str = re.sub(r'[\W_]+', '', token).lower()
                if term_str not in stop_words:
                    tokens.append((i, j, term_str))
        self.TOKEN = pd.DataFrame(tokens, columns=['doc_id','token_num','term_str'])\
            .set_index(['doc_id','token_num'])

        # Extract vocabulary
        self.VOCAB = self.TOKEN.term_str.value_counts().to_frame('n')
        self.VOCAB.index.name = 'term_str'    
        

### Gibbs Sampler

We sample each document and word combination in the BOW table. In each case,
we are looking for two values:

* the topic with which a word has been most frequently labeled
* the topic with which the document has the most labeled words

We combine these values in order to align the label of the current word with the rest of the data.\
If a the topic is highly associated with both the word and the document, then that topic will get a high value.

Note that all that is going on here is a sorting operation -- the random assignment does not predict anything.\
Instead, we are just gathering words under topics and topics under documents.

**From Darling 2011:**
<hr />
<div style="float:left;">
<img src="images/gibbs-algo-text.png" width="650px" />
<img src="images/gibbs-algo.png" width="650px" />
</div>

In [72]:
class GibbsSampler():
    
    n_topics:int = 10
    n_iters:int = 100
    a:float = 1.
    b:float = .1

    # See Griffiths and Steyvers 2004
    #     a = 1 # 50 / n_topics
    #     b = .1 # 200 / W

    def __init__(self, corpus:Corpus):
        self.corpus = corpus
        self.N = len(corpus.TOKEN)
        self.W = len(corpus.VOCAB)
        
    def generate_model(self):
        
        # Create topics table
        zcols = range(self.n_topics) 
        self.topics = pd.DataFrame(index=zcols)

        # Randomly assign topics to toknes
        self.corpus.TOKEN['topic_id'] = self.topics.sample(self.N, replace=True).index

        # Create one-hot-encoding columns for easier computation
        self.Z = pd.concat([self.corpus.TOKEN, pd.get_dummies(self.corpus.TOKEN.topic_id)], axis=1)
        
        # Iterate
        for x in tqdm(range(self.n_iters)):
            
            # Loop through tokens
            for i in self.Z.index:                
                
                # Get row elements
                d = i[0] # Current document
                z = self.Z.loc[i, zcols].idxmax() # Current assigned topic
                w = self.Z.loc[i].term_str # Current term
                
                # Zero out the current topic assignment
                self.Z.loc[i, z] = 0

                pz = [] # Weigths of topics for words
                c = self.b * self.W # Precompute 
                
                # Look through topics
                for k in zcols:
 
                    # Number of words assigned to topic k in the document
                    n_dk = self.Z.loc[d][k].sum()

                    # Number of times word w is assigned to topic k
                    n_kw = self.Z.groupby('term_str')[k].sum()

                    # Number of times any word is assigned to topic k
                    n_k = self.Z[k].sum()

                    # Generate probalities based on current state of everything else
                    # Note formula involves a LOCAL and a GLOBAL measure, kinda like TF-IDF
                    p = (n_dk + self.a) * ((n_kw + self.b) / (n_k + c))   

                    # Add to weights list
                    pz.append(p)
                                    
                # Sample the new topic assignment from the weights
                z2 = pd.DataFrame(pz).T.loc[w].sample().index[0]
                
                # Update the token assignment (redundantly)
                self.Z.loc[i, z2] = 1
                self.Z.loc[i, 'topic_id'] = z2
        
        # Create topic model tables
        self.topics['n_tokens'] = self.Z.value_counts('topic_id')
        self.theta = self.Z.value_counts(['doc_id','topic_id']).unstack(fill_value=0)
        self.phi = self.Z.value_counts(['term_str','topic_id']).unstack(fill_value=0)
        self.theta = (self.theta.T / self.theta.T.sum()).T
        
        # Get top words for each topic
        self.topics['top_terms'] = self.topics.apply(lambda x: self.phi[x.name].sort_values(ascending=False)\
                                                     .head().index.to_list(), 1)   
        

## Demo 1

We use a toy example to see if the method works.\
Because our codd is not vert efficient, we just 

### Data

A small F1 corpus.

In [22]:
raw_docs = """
I ate a banana and a spinach smoothie for breakfast.
I like to eat broccoli and bananas.
Chinchillas and kittens are cute.
My sister adopted a kitten yesterday.
Look at this cute hamster munching on a piece of broccoli.
""".split("\n")[1:-1]

### Process

In [23]:
pd.options.mode.chained_assignment = None

In [24]:
corpus1 = Corpus(raw_docs)

In [73]:
model1 = GibbsSampler(corpus1)
model1.n_topics = 4
model1.n_iters = 200
model1.generate_model()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:27<00:00,  7.36it/s]


In [74]:
model1.topics

,n_tokens,top_terms
0,5,"[adopted, broccoli, cute, chinchillas, yesterday]"
1,5,"[kittens, banana, sister, munching, look]"
2,5,"[kitten, like, bananas, breakfast, cute]"
3,7,"[spinach, smoothie, broccoli, piece, eat]"


## Experiment

In [90]:
zcols = range(model1.n_topics)

In [150]:
s = model1.Z.sample()
d = s.index[0][0]
w = s.term_str.values[0]
z = s.topic_id.values[0]

In [151]:
d, w, z

(0, 'ate', 3)

In [152]:
n_dk = model1.Z.loc[d, zcols].sum().to_frame('n')
n_dk.index.name = 'topic_id'

In [153]:
n_dk

,n
topic_id,
0,0
1,1
2,1
3,3


In [174]:
n_kw =  model1.Z.groupby('term_str').sum()

In [175]:
n_kw

,topic_id,0,1,2,3
term_str,,,,,
adopted,0,1,0,0,0
ate,3,0,0,0,1
banana,1,0,1,0,0
bananas,2,0,0,1,0
breakfast,2,0,0,1,0
broccoli,3,1,0,0,1
chinchillas,0,1,0,0,0
cute,2,1,0,1,0
eat,3,0,0,0,1


## Demo 2

### Data

In [54]:
some_documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]
raw_docs2  = [' '.join(item) for item in some_documents]

### Process

In [55]:
corpus2 = Corpus(raw_docs2)

In [29]:
model2 = GibbsSampler(corpus2)

In [59]:
model2.n_topics = 10
model2.n_iters = 200
model2.generate_model()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [03:19<00:00,  1.00it/s]


In [60]:
model2.topics

,n_tokens,top_terms
0,13,"[probability, statistics, regression, storm, d..."
1,10,"[networks, big, data, pandas, postgres]"
2,9,"[scipy, learning, c, cassandra, data]"
3,5,"[python, neural, mapreduce, scikitlearn, r]"
4,9,"[artificial, deep, support, neural, mahout]"
5,8,"[libsvm, cassandra, big, hadoop, hbase]"
6,9,"[regression, c, trees, statsmodels, statistics]"
7,8,"[java, hadoop, mongodb, pandas, scikitlearn]"
8,5,"[artificial, python, numpy, libsvm, big]"
9,6,"[java, mongodb, data, databases, postgres]"


In [61]:
corpus2.DOC.join(model2.theta).style.background_gradient(cmap='GnBu', high=.5, axis=None)

,doc_str,0,1,2,3,4,5,6,7,8,9
doc_id,,,,,,,,,,,
0,Hadoop Big Data HBase Java Spark Storm Cassandra,0.125000,0.125000,0.125000,0.000000,0.000000,0.125000,0.000000,0.250000,0.000000,0.250000
1,NoSQL MongoDB Cassandra HBase Postgres,0.000000,0.200000,0.000000,0.000000,0.200000,0.400000,0.000000,0.000000,0.000000,0.200000
2,Python scikit-learn scipy numpy statsmodels pandas,0.000000,0.166667,0.166667,0.333333,0.000000,0.000000,0.166667,0.000000,0.166667,0.000000
3,R Python statistics regression probability,0.600000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000
4,machine learning regression decision trees libsvm,0.166667,0.166667,0.166667,0.000000,0.000000,0.166667,0.333333,0.000000,0.000000,0.000000
5,Python R Java C++ Haskell programming languages,0.285714,0.142857,0.142857,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.142857
6,statistics probability mathematics theory,0.250000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000
7,machine learning scikit-learn Mahout neural networks,0.166667,0.000000,0.000000,0.166667,0.166667,0.000000,0.166667,0.333333,0.000000,0.000000
8,neural networks deep learning Big Data artificial intelligence,0.125000,0.250000,0.000000,0.000000,0.250000,0.125000,0.125000,0.000000,0.125000,0.000000


In [65]:
model2.topics.sort_values('n_tokens', ascending=False).style.bar()

,n_tokens,top_terms
0,13,"['probability', 'statistics', 'regression', 'storm', 'decision']"
1,10,"['networks', 'big', 'data', 'pandas', 'postgres']"
2,9,"['scipy', 'learning', 'c', 'cassandra', 'data']"
4,9,"['artificial', 'deep', 'support', 'neural', 'mahout']"
6,9,"['regression', 'c', 'trees', 'statsmodels', 'statistics']"
5,8,"['libsvm', 'cassandra', 'big', 'hadoop', 'hbase']"
7,8,"['java', 'hadoop', 'mongodb', 'pandas', 'scikitlearn']"
9,6,"['java', 'mongodb', 'data', 'databases', 'postgres']"
3,5,"['python', 'neural', 'mapreduce', 'scikitlearn', 'r']"
8,5,"['artificial', 'python', 'numpy', 'libsvm', 'big']"
